In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
!wget --show-progress --continue -O /content/oddesey.txt http://www.gutenberg.org/files/2229/2229-0.txt

--2021-01-28 21:21:34--  http://www.gutenberg.org/files/2229/2229-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
!head -n5 /content/oddesey.txt
!echo '...'
!shuf -n5 /content/oddesey.txt

﻿The Project Gutenberg eBook of Faust: Der Tragödie erster Teil, by Johann Wolfgang von Goethe

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
...
Ja, singe, singe nur und lob und rühme sie!
Man läuft euch bei dem ersten Blick davon.
So wag ich’s, dich hinwegzutragen.
Ich will von keinem Gruße wissen,
Lege dich hinter den Ofen nieder,


In [ ]:
filepath = '/content/oddesey.txt'

In [ ]:
with open(filepath) as f:
  Oddesey_text = f.read()

In [ ]:
Oddesey_text

'\ufeffThe Project Gutenberg eBook of Faust: Der Tragödie erster Teil, by Johann Wolfgang von Goethe\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: Faust: Der Tragödie erster Teil\n\nAuthor: Johann Wolfgang von Goethe\n\nRelease Date: June, 2000 [eBook #2229]\n[Most recently updated: January 19, 2021]\n\nLanguage: German\n\nCharacter set encoding: UTF-8\n\nProduced by: Michael Pullen\n\n*** START OF THE PROJECT GUTENBERG EBOOK FAUST: DER TRAGÖDIE ERSTER TEIL ***\n\n\n\n\nFaust:\nDer Tragödie erster Teil\n\nby Johann Wolfgang von Goethe\n\n\nContents\n\n Zueignung\n Vorspiel auf 

In [ ]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level = True)

In [ ]:
tokenizer.fit_on_texts([Oddesey_text])

In [ ]:
[encode] = np.array(tokenizer.texts_to_sequences([Oddesey_text]))

In [ ]:
seq = tokenizer.texts_to_sequences(['king'])
seq

[[24, 4, 3, 17]]

In [ ]:
tokenizer.texts_to_sequences(['god'])

[[17, 16, 10]]

In [ ]:
tokenizer.sequences_to_texts([[17, 16, 10]])

['g o d']

Each letter is mapped to a number.

Now we make a model to detect pos and neg film reviews.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name = 'imdb_reviews',
    split = ('train[:60%]', 'train[:20%]', 'test'),
    as_supervised = True
)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [ ]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable = True) # We can decide if to train them or not
# hub_layer is an embedder.
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
# In keras we 'declare' a model.
model = tf.keras.Sequential()
model.add(hub_layer)
# The activaion parameter is just a name (string)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
# The problem is to decide if the review is pos or neg. Then it only needs one output.
model.add(tf.keras.layers.Dense(1))
model.summary()
# Params are weights and biases.
# The activation functions are not defined yet. This is just building the model.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# logits is same as sigmoid.
model.compile(optimizer= 'Adam', loss = tf.keras.losses.BinaryCrossentropy(from_logits = True), metrics = ['accuracy'])

In [ ]:
# history contains all the loss calculations, the weights etc. during the training.
# We divide training data into 512 batches, 10000 each and 
history = model.fit(train_data.shuffle(10000).batch(512), epochs = 20, validation_data = validation_data.batch(512), verbose = 1)

Epoch 1/20
30/30 [==============================] - 3s 77ms/step - loss: 0.1543 - accuracy: 0.9464 - val_loss: 0.1464 - val_accuracy: 0.9488
Epoch 2/20
30/30 [==============================] - 3s 79ms/step - loss: 0.1434 - accuracy: 0.9503 - val_loss: 0.1360 - val_accuracy: 0.9518
Epoch 3/20
30/30 [==============================] - 3s 76ms/step - loss: 0.1348 - accuracy: 0.9534 - val_loss: 0.1273 - val_accuracy: 0.9590
Epoch 4/20
30/30 [==============================] - 3s 76ms/step - loss: 0.1254 - accuracy: 0.9580 - val_loss: 0.1184 - val_accuracy: 0.9638
Epoch 5/20
30/30 [==============================] - 3s 77ms/step - loss: 0.1164 - accuracy: 0.9631 - val_loss: 0.1089 - val_accuracy: 0.9664
Epoch 6/20
30/30 [==============================] - 3s 79ms/step - loss: 0.1088 - accuracy: 0.9660 - val_loss: 0.1030 - val_accuracy: 0.9628
Epoch 7/20
30/30 [==============================] - 3s 76ms/step - loss: 0.1016 - accuracy: 0.9682 - val_loss: 0.0931 - val_accuracy: 0.9728
Epoch 8/20
30

In [ ]:
# In case of this model it is not very good to have very high accuracy. It works poorly when deployed.
results = model.evaluate(test_data.batch(512), verbose = 2)

49/49 - 2s - loss: 0.4434 - accuracy: 0.8502


In [ ]:
model.metrics

In [ ]:
for name, value in zip(model.metrics_names, results):
  print('%s: %.3f' % (name, value))


loss: 0.443
accuracy: 0.850


In [ ]:
model.predict(['I hated it, it reeked, I loathe this movie'])

array([[-1.6690459]], dtype=float32)

In [ ]:
# We turn values to true and false with a threshold of 0.5.
(model.predict(['I hated it, it reeked, I loathe this movie']) > 0.5).astype('int32')

array([[0]], dtype=int32)

In [ ]:
# This time it decided that it is a positive review.
(model.predict(['I had it, it reeked, I love this movie']) > 0.5).astype('int32')

array([[1]], dtype=int32)

In [ ]:
(model.predict(['I can not forget it']) > 0.5).astype('int32')

array([[0]], dtype=int32)